In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", top_k=None)

# Test the model with a sentence
result = sentiment_analyzer("Trump improves GDP by 5% and Kamala increases homelessness by 5%")
print(result)

In [ ]:
model_name ="lxyuan/distilbert-base-multilingual-cased-sentiments-student"

tokenizer = AutoTokenizer.from_pretrained(model_name)

input_text = "Trump improves GDP by 5% and Kamala increases homelessness by 5%."
encoded_input = tokenizer(input_text)
#encoded_input["input_ids"]

In [ ]:
tokenizer.convert_ids_to_tokens(29846)

In [ ]:
#tokenizer.convert_ids_to_tokens(encoded_input["input_ids"])

In [ ]:
# Load the original tokenizer
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add the new token to the tokenizer
new_tokens = ["Kamala"]
num_added_toks = tokenizer.add_tokens(new_tokens)

# Check how many tokens were added
print(f"Added {num_added_toks} tokens.")

# Test encoding after adding the new token
input_text = "Trump improves GDP by 5% and Kamala increases homelessness by 5%."
encoded_input = tokenizer(input_text)
print(encoded_input["input_ids"])

# Convert back to tokens to see the result
print(tokenizer.convert_ids_to_tokens(encoded_input["input_ids"]))


In [ ]:
#encoded_input["input_ids"]

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Resize the model’s token embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer.convert_ids_to_tokens(119547)

In [ ]:
tokenizer.vocab_size

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
#model

In [ ]:
tokenizer.convert_tokens_to_ids("Kamala")